In [1]:
import sys

# install kfp (https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.html)
!{sys.executable} -m pip install --user --upgrade -q kfp==0.4.0

# Download skaffold and set it executable.
!curl -Lo skaffold https://storage.googleapis.com/skaffold/releases/latest/skaffold-linux-amd64 && chmod +x skaffold && mv skaffold /home/jupyter/.local/bin/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 44.6M  100 44.6M    0     0   112M      0 --:--:-- --:--:-- --:--:--  111M


In [2]:
# !{sys.executable} -m pip install --user --upgrade -q tfx==0.22.0
# !{sys.executable} -m pip install --user --upgrade -q tfx==0.21.4
!python3 -c "import tfx; print('TFX version: {}'.format(tfx.__version__))"

TFX version: 0.22.0


In [3]:
# Set `PATH` to include user python binary directory and a directory containing `skaffold`.
PATH=%env PATH
%env PATH={PATH}:/home/jupyter/.local/bin

env: PATH=/usr/local/cuda/bin:/opt/conda/bin:/opt/conda/condabin:/usr/local/bin:/usr/bin:/bin:/usr/local/games:/usr/games:/home/jupyter/.local/bin


In [4]:
# Read GCP project id from env.
shell_output=!gcloud config list --format 'value(core.project)' 2>/dev/null
GCP_PROJECT_ID=shell_output[0]
print("GCP project ID:" + GCP_PROJECT_ID)

GCP project ID:nitroml-brain-xgcp


# Set KFP Cluster End point

In [5]:
# This refers to the KFP cluster endpoint
ENDPOINT='ee1a2cabbbc2f13-dot-us-east1.pipelines.googleusercontent.com' # Enter your ENDPOINT here.
if not ENDPOINT:
    from absl import logging
    logging.error('Set your ENDPOINT in this cell.')

In [6]:
# Docker image name for the pipeline image 
IMAGE_NAME = 'nitroml_benchmark4'
CUSTOM_TFX_IMAGE='gcr.io/' + GCP_PROJECT_ID + '/' + IMAGE_NAME

In [7]:
import os
PROJECT_DIR=os.path.join(os.path.expanduser("~"), "AIHub" , 'nitroml')
%cd {PROJECT_DIR}

/home/jupyter/AIHub/nitroml


In [8]:
from examples import config
PIPELINE_NAME=config.PIPELINE_NAME

In [9]:
PIPELINE_NAME

'titanic_new2'

## Run the tfx pipeline

In [10]:
!tfx pipeline create  \
--pipeline-path=examples/kubeflow_dag_runner.py \
--endpoint={ENDPOINT} \
--build-target-image={CUSTOM_TFX_IMAGE}

CLI
Creating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
No local setup.py, copying the directory and configuring the PYTHONPATH.
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
['/home/jupyter/AIHub/nitroml/examples', '/home/jupyter/AIHub/nitroml/examples/..', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '/home/jupyter/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages']
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
I0626 19:15:40.794073 139838116447616 dataset_info.py:361] Load dataset info from gs://artifacts.nitroml-brain-

In [11]:
!tfx run create --pipeline-name={config.PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: titanic_new2
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: titanic_new2
+-----------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------+
| pipeline_name   | run_id                               | status   | created_at                | link                                                                                                                    |
+=================+======================================+==========+===========================+=========================================================================================================================+
| titanic_new2    | a8e0681d-a156-4810-b8a0-2e74cf2c6157 |          | 2020-06-26T19:15:52+00:00 | http://ee1a2cabbbc2f13-dot-us-east1.pipelines.googleusercontent.com/#/run

In [10]:
# Update the pipeline
!tfx pipeline update \
--pipeline-path=examples/kubeflow_dag_runner.py \
--endpoint={ENDPOINT}

CLI
Updating pipeline
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Reading build spec from build.yaml
Use skaffold to build the container image.
/home/jupyter/.local/bin/skaffold
New container image is built. Target image is available in the build spec file.
['/home/jupyter/AIHub/nitroml/examples', '/home/jupyter/AIHub/nitroml/examples/..', '/opt/conda/lib/python37.zip', '/opt/conda/lib/python3.7', '/opt/conda/lib/python3.7/lib-dynload', '/home/jupyter/.local/lib/python3.7/site-packages', '/opt/conda/lib/python3.7/site-packages']
/home/jupyter/.local/lib/python3.7/site-packages/tfx/orchestration/data_types.py:191: UserWarning: RuntimeParameter is only supported on KubeflowDagRunner currently.
  warnings.warn('RuntimeParameter is only supported on KubeflowDagRunner '
I0626 19:47:13.368769 140310186665344 dataset_info.py:361] Load dataset info from gs://artifacts.nitroml-brain-xgcp.appspot.com/tensorflow-datasets/titanic/2.0.0
I0626 19:47:14.028251 

In [11]:
!tfx run create --pipeline-name {PIPELINE_NAME} --endpoint={ENDPOINT}

CLI
Creating a run for pipeline: titanic_new2
Detected Kubeflow.
Use --engine flag if you intend to use a different orchestrator.
Run created for pipeline: titanic_new2
+-----------------+--------------------------------------+----------+---------------------------+-------------------------------------------------------------------------------------------------------------------------+
| pipeline_name   | run_id                               | status   | created_at                | link                                                                                                                    |
+=================+======================================+==========+===========================+=========================================================================================================================+
| titanic_new2    | c336ddf4-1347-42f4-822d-e2d20ec722c6 |          | 2020-06-26T19:47:22+00:00 | http://ee1a2cabbbc2f13-dot-us-east1.pipelines.googleusercontent.com/#/run

In [37]:
tfx_image = os.environ.get('KUBEFLOW_TFX_IMAGE', None)

In [38]:
print (tfx_image)

None
